In [1]:
import numpy as np
import pandas as pd

In [2]:
smd_train_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_train.npy")
smd_test_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_test.npy")
smd_label_data = np.load("/home/joono/Anomaly-Transformer/dataset/SMD/SMD_test_label.npy")

In [3]:
smd_train_data.shape, smd_test_data.shape, smd_label_data.shape

((708405, 38), (708420, 38), (708420,))

In [4]:
sample_df = pd.read_csv("/home/joono/AnomalyEnvDetector/dataset/SmartFarm/SmartFarm_tomato_env_sensor_data.csv")
sample_df_flag = sample_df.copy()

for column in sample_df.columns:
    if column == "index" or column == "facilityId" or column == "measDate": continue
    # Create a new column with '_flag' suffix that checks for NaN
    sample_df_flag[column + '_NaN'] = sample_df[column].isna().astype(float)
    sample_df_flag[column] = pd.to_numeric(sample_df_flag[column], errors="coerce")

sample_df_flag_fillnan = sample_df_flag.fillna(0)
np_time_stamps = sample_df_flag_fillnan["measDate"].to_numpy()

sample_df_flag_fillnan_onlydata = sample_df_flag_fillnan.drop(columns=['measDate'], inplace=False)
np_df_flat_fillna = sample_df_flag_fillnan_onlydata.to_numpy()

In [5]:
print(sample_df_flag.describe())

               index             CI             DT             HI  \
count  503695.000000  480401.000000  178377.000000  492211.000000   
mean   251847.000000     507.303395      20.043280      91.506781   
std    145404.366257     385.668418      34.910576      42.341709   
min         0.000000       0.000000       0.000000       0.000000   
25%    125923.500000     320.600000      12.000000      66.620000   
50%    251847.000000     426.100000      16.000000      88.970000   
75%    377770.500000     589.700000      22.000000      97.920000   
max    503694.000000    3478.100000     655.000000     300.000000   

                 IR             TI             HL             PL  \
count  13315.000000  499482.000000  451466.000000  104944.000000   
mean     177.072472      21.087261      30.271563       1.195280   
std      332.493118      10.887839      41.023416       2.446872   
min        0.000000     -24.000000    -324.000000       0.000000   
25%        0.000000      14.100000    

In [6]:
sample_df.to_csv("results_input.csv")
sample_df_flag.to_csv("results_flag.csv")
sample_df_flag_fillnan.to_csv('results_flag_fillna.csv')

In [25]:
from datetime import datetime

def get_hour_index_of_year(timestamp):
    # Parse the timestamp string to a datetime object
    date_time_obj = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    
    # Calculate the day of the year from the datetime object
    day_of_year = date_time_obj.timetuple().tm_yday
    
    # Get the hour from the datetime object
    hour_of_day = date_time_obj.hour
    
    # Calculate the hour index within the year (0-indexed)
    hour_index_of_year = (day_of_year - 1) * 24 + hour_of_day
    
    return hour_index_of_year

In [31]:
df = pd.read_csv("/home/joono/AnomalyEnvDetector/results_flag_fillna.csv")
time_indices = df["measDate"].apply(get_hour_index_of_year).to_numpy()

# time_stamps = df["measDate"].to_numpy()
df.drop(columns=["index", "facilityId", "measDate", "Unnamed: 0"], inplace=True) # Unnamed: 0 은 pandas로 pivoting 하는 과정에서 생김
sensor_datas = df.to_numpy()

print(len(time_indices), time_indices)
print(len(sensor_datas), sensor_datas.shape, sensor_datas[0])

503695 [6552 6553 6554 ... 8664 8665 8666]
503695 (503695, 18) [508.    16.    87.33   0.    19.67   0.     0.     0.     0.     0.
   0.     0.     1.     0.     0.     1.     1.     0.  ]


In [36]:
dataset = np.concatenate((time_indices.reshape((len(time_indices), 1)), sensor_datas), axis=1)
dataset.shape, dataset[0]

((503695, 19),
 array([6.552e+03, 5.080e+02, 1.600e+01, 8.733e+01, 0.000e+00, 1.967e+01,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        0.000e+00]))

In [37]:
np.savez("/home/joono/AnomalyEnvDetector/dataset/SmartFarm/smartfarm_train.npz", dataset)

In [62]:
def positional_encoding(time_steps, dimension):
    assert dimension % 2 == 0, "Dimension must be even."
    position = np.arange(time_steps).reshape(-1, 1)
    div_term = np.exp(np.arange(0, dimension, 2) * -(np.log(10000.0) / dimension))
    pe = np.zeros((time_steps, dimension))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    return pe

# Specify dimension based on your model requirements
dimension = 14 # num of predefined columns (We will use only up to 13)
pos_encodings = positional_encoding(365 * 24, dimension) # 8760 = 365 days * 24 hours

In [64]:
for i, time_stamp in enumerate(np_time_stamps):
    tidx = get_hour_index_of_year(time_stamp)
    np_df_flat_fillna[i, :13] += pos_encodings[tidx, :13]    

In [69]:
np_df_flat_fillna[4]

array([-0.82183501, -0.56972556, 95.26263333, -0.93576158,  0.42414733,
        0.9055932 , -0.89521588,  0.44563274,  0.98335725,  0.18168246,
        0.36383781,  0.93146232,  0.09973343,  1.        ,  1.        ,
        0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.        ])

In [58]:
pos_encodings[0]

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [70]:
np.savez("sample_data.npz", np_df_flat_fillna)

In [71]:
np_df_flat_fillna.shape

(2083, 26)

In [5]:
data = np.load("/home/joono/Anomaly-Transformer/dataset/SmartFarm/smartfarm_train.npz")

In [13]:
data['arr_0']

array([[ 6.06276017e+02,  9.94364266e-01,  9.37605845e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 8.94010170e-01,  4.48046667e-01,  9.60588643e+01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 6.46360054e+02, -5.10202971e-01,  9.55614104e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [ 5.67344369e+02,  9.14596145e-01,  9.94832508e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 4.36178087e+02,  1.53894037e-01,  9.78326470e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 6.75573363e+02, -7.48297539e-01,  9.91062075e+01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00]])